## Harmony Py Library
### Job Results Iterator Example

In [ ]:
import sys
sys.path.append('..')

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

import concurrent.futures
import os
from harmony import BBox, Client, Collection, Request, Environment


In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77)
)


In [ ]:
# submit an async request for processing and return the job_id
job_id = harmony_client.submit(request)
job_id


In [ ]:
# We can iterate over the results which will cause them to download in parallel while the job is running
# the first argument to `iterator()` is the ID of the job we want
# the second argument is the path where we want to download the files
# the third argument is whether or not we want to overwrite files - this defaults to False which
# speeds up processing by not re-downloading files that have already been downloaded
iter = harmony_client.iterator(job_id, '', True)
# grab all the futures that are downloading the files
futures = list(map(lambda x: x['path'], iter))
# wait for all the futures to complete
(done_futures, _) = concurrent.futures.wait(futures)
#iterate of the futures to get the file paths
for f in done_futures:
  path = f.result()
  print(path)

In [ ]:
# submit another async request for processing and return the job_id
job_id = harmony_client.submit(request)
job_id

In [ ]:
# We can iterator over the results without waiting for the job to complete before downloading
# the first results. This is slower because we don't start downloading a file until we have
# finished downloading the previous file, but it is still faster than `download_all` because
# `download_all` waits for the job to complete before starting downloads. Also, this approach
# allows us to peak at the files as they come in, rather than waiting for everything to finish
# downloading as we did above.
iter = harmony_client.iterator(job_id, '', True)
for result in iter:
  # print the file name
  path_future = result['path']
  path = path_future.result()
  print(path)